## Поиск кратчайшего пути между двумя вершинами графа с помощью алгоритма Дейкстры и муравьиной оптимизации

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import random
from numpy.random import choice as np_choice
from tqdm import tqdm

In [2]:
# возвращает матрицу размером NxN с неотрицательными весами
def generate_graph(N):
    matrix = np.zeros(N*N,dtype='float').reshape((N,N))
    # указываем, что вершины не связаны сами с собой
    for i in range(N): matrix[i, i] = np.float('inf')
    # создаем "каркас", чтоб точно получилась ровно одна компонента связности    
    path = list(range(N))
    random.shuffle(path)
    for v1, v2 in zip(path, path[1:]):
        matrix[v1, v2] = matrix[v2, v1] = random.random() + 0.3
    # добавляем случайное количество новых ребер
    for i in range(N):
        for j in range(i+1, N):
            if np.isclose(matrix[i, j], 0):
                matrix[i, j] = matrix[j, i] = np.float('inf') if random.random() < 0.8 else random.random() + 0.5
    return matrix

In [3]:
N = 500
start, end = random.randint(0, N-1), random.randint(0, N-1)
M = generate_graph(N)
M.shape

(500, 500)

In [38]:
def dijikstra(M, N, s, e):
    # длины путей
    d = [np.float('inf') for _ in range(N)]
    d[s] = 0
    # метки посещенных вершин
    labels  = [1 for _ in range(N)] 
  
    while True:
        minindex = -1
        min = np.float('inf')
        for i in range(N):
            if labels[i] and d[i] < min:
                min = d[i]
                minindex = i
        
        if minindex >= 0:
            for i in range(N):
                if M[minindex][i] < np.float('inf'):
                    temp = min + M[minindex][i]
                    if temp < d[i]: d[i] = temp
            labels[minindex] = 0;
        else: break
    
    return d[e]

In [39]:
dijikstra(M, N, 0, 200)

0.96530635694635625

In [44]:
# возвращает средний путь
def ant_colony(M, N, s, e, num_of_ants=200, alpha=0.7, beta=2, times=300, p_=0.1):
    pheromones = np.ones(M.shape)
    # вероятность перехода между s и i
    probs = np.ones(M.shape)
    
    # муравьи ищут путь между s и e
    # возвращают все эти пути
    def ant_search(s, e):
        paths = []
        
        # инициализируем вероятности
        for i in range(N):
            for j in range(N):
                probs[i, j] = probs[j, i] = pheromones[j, i]**alpha * (((1/M[j, i])**beta) if M[j, i] < float('inf') else 0)
        
        # для каждого муравья находим путь
        for i in range(num_of_ants):
            current, pher, path = s, 1, [s] 
            while current != e and pher > 0:
                next = np.random.choice(list(range(N)),1,p=probs[current]/sum(probs[current]))[0]
                pher -= p_
                path.append(next)
                current = next
            if pher > 0:
                paths.append(path) 
        return paths    
    
    # сколько эпох
    for t in tqdm(range(times)):
        # находим пути
        paths = ant_search(s, e)
        # print(len(paths))
        path_costs = []
        # обновляем феромоны
        for p in paths:
            path_cost = sum(M[x,y] for x, y in zip(p, p[1:]))
            path_costs.append(path_cost)
            for a,b in zip(p, p[1:]):
                pheromones[a,b] = (1-p_)*pheromones[a,b] + 1000/path_cost
        #print(np.mean(path_costs))
    
    
    paths = []
     # для каждого муравья находим путь
    for i in range(num_of_ants):
        current, pher, path = s, 1, [s] 
        while current != e and pher > 0:
            next = np.random.choice(list(range(N)),1,p=probs[current]/sum(probs[current]))[0]
            pher -= p_
            path.append(next)
            current = next
        if pher > 0:
            paths.append(path)
        
    
    path_costs = [sum(M[x,y] for x, y in zip(p, p[1:])) for p in paths]
    print(paths[np.argmin(path_costs)], min(path_costs))
    return np.mean(path_costs), np.mean([len(p) for p in paths]), sum(np.isclose(path_costs, dijikstra(M, N, s, e)))

        

In [45]:
ant_colony(M, N, 0, 200)

100%|████████████████████████████████████████| 300/300 [05:47<00:00,  1.16s/it]


[0, 223, 242, 404, 101, 200] 2.74055549825


(4.7571949960592148, 7.5625, 0)